In [ ]:
# 국토 교통부의 자료를 가져와서, Path와 함께 Key를 정의해서 해당 Code를 돌려주면 된다.
# 1. 국토 교통부의 CSV 파일을 메모장으로 열어서 UTF-8로 변환해야 한다.
# 2. 국토 교통부의 CSV 파일의 윗 Header의 내용은 삭제해야 한다.
# 3. 다음의 Code를 거치면, HouseInfo에서 법정동 Code와 경도, 위도가 제외된 결과의 CSV가 반환된다.
# 4. 해당 CSV를 바탕으로 추가로 KaKao의 주소 좌표 변환 API를 통해 경도, 위도 그리고 법정동 Code를 가져와야 한다.

import pandas as pd
import csv

# Define paths to the raw CSV files
raw_csv_files = {
    'apartment_sales_2023': 'apartment_sales_2023.csv',
    'apartment_sales_2024': 'apartment_sales_2024.csv',
    'multi_house_sales_2023': 'multi_house_sales_2023.csv',
    'multi_house_sales_2024': 'multi_house_sales_2024.csv',
    'single_house_sales_2023': 'single_house_sales_2023.csv',
    'single_house_sales_2024': 'single_house_sales_2024.csv',
    'officetels_sales_2023': 'officetels_sales_2023.csv',
    'officetels_sales_2024': 'officetels_sales_2024.csv',
    'apartment_rent_2023': 'apartment_rent_2023.csv',
    'apartment_rent_2024': 'apartment_rent_2024.csv',
    'multi_house_rent_2023': 'multi_house_rent_2023.csv',
    'multi_house_rent_2024': 'multi_house_rent_2024.csv',
    'single_house_rent_2023': 'single_house_rent_2023.csv',
    'single_house_rent_2024': 'single_house_rent_2024.csv',
    'officetels_rent_2023': 'officetels_rent_2023.csv',
    'officetels_rent_2024': 'officetels_rent_2024.csv'
}

# Define new CSV file paths
new_house_deal_csv = 'house_deal.csv'
new_house_info_csv = 'house_info.csv'

# Initialize counters
house_code_counter = 11110000000001
deal_code_counter = 1
seen_addresses = set()

# Function to try different encodings
def read_csv_with_encoding(filepath):
    encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
    for encoding in encodings:
        try:
            return pd.read_csv(filepath, encoding=encoding)
        except UnicodeDecodeError:
            continue
    print(f"Error: Unable to decode file {filepath} with tried encodings.")
    return None

# Define the function to process and convert the data
def process_csv(raw_csv_files):
    global house_code_counter, deal_code_counter, seen_addresses
    house_deals = []
    house_infos = []

    for key, filepath in raw_csv_files.items():
        df = read_csv_with_encoding(filepath)
        if df is None:
            continue
        
        # Print column names for debugging
        print(f"Columns in file {filepath}: {df.columns.tolist()}")
        
        # Handle possible column name variations
        col_계약년월 = '계약년월' if '계약년월' in df.columns else '계약년월_대체'
        col_계약일 = '계약일' if '계약일' in df.columns else '계약일_대체'
        col_거래금액 = '거래금액(만원)' if '거래금액(만원)' in df.columns else '거래금액_대체'
        col_전용면적 = '전용면적(㎡)' if '전용면적(㎡)' in df.columns else '계약면적(㎡)'
        col_보증금 = '보증금(만원)' if '보증금(만원)' in df.columns else '보증금_대체'
        col_월세금 = '월세금(만원)' if '월세금(만원)' in df.columns else '월세금_대체'
        
        # Check required columns
        required_columns = ['NO', col_계약년월, col_계약일]
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: Missing columns {missing_columns} in file {filepath}")
            continue
        
        # Process each row in the DataFrame
        for _, row in df.iterrows():
            house_type = {
                'apartment_sales_2023': 1,
                'apartment_sales_2024': 1,
                'apartment_rent_2023': 1,
                'apartment_rent_2024': 1,
                'multi_house_sales_2023': 2,
                'multi_house_sales_2024': 2,
                'multi_house_rent_2023': 2,
                'multi_house_rent_2024': 2,
                'single_house_sales_2023': 3,
                'single_house_sales_2024': 3,
                'single_house_rent_2023': 3,
                'single_house_rent_2024': 3,
                'officetels_sales_2023': 4,
                'officetels_sales_2024': 4,
                'officetels_rent_2023': 4,
                'officetels_rent_2024': 4,
            }[key]
            
            if 'sales' in key:
                deal_type = 1
                deal_amount = row[col_거래금액] if col_거래금액 in row else None
                deposit_amount = None
                monthly_amount = None
            elif 'rent' in key:
                deal_type = 2 if row['전월세구분'] == '전세' else 3
                deal_amount = None
                deposit_amount = row[col_보증금] if col_보증금 in row else None
                monthly_amount = row[col_월세금] if col_월세금 in row else None

            deal_year = str(row[col_계약년월])[:4]
            deal_month = str(row[col_계약년월])[4:6]
            deal_day = row[col_계약일]
            area = row[col_전용면적] if col_전용면적 in row else None
            floor = row['층'] if '층' in row else None

            # Generate deal_code
            deal_code = f"11110{house_type}{deal_type}{deal_code_counter:07}"
            deal_code_counter += 1

            house_deal = [
                deal_code,
                deal_type,
                deal_amount,
                deposit_amount,
                monthly_amount,
                deal_year,
                deal_month,
                deal_day,
                area,
                floor,
                house_code_counter,
                house_type
            ]
            house_deals.append(house_deal)

            address = f"{row['시군구']} {row['도로명']}"
            if address not in seen_addresses:
                seen_addresses.add(address)
                house_info = [
                    house_code_counter,
                    house_type,
                    row['건축년도'] if '건축년도' in row else None,
                    row['시군구'].split(' ')[2] if '시군구' in row else None,
                    row['단지명'] if '단지명' in row else row['건물명'] if '건물명' in row else None,
                    row['시군구'].split(' ')[1] if '시군구' in row else None,
                    row['시군구'].split(' ')[2] if '시군구' in row else None,
                    row['도로명'] if '도로명' in row else None,
                    row['본번'] if '본번' in row else None,
                    row['부번'] if '부번' in row else None,
                    address,
                    None,  # lng - needs geocoding
                    None  # lat - needs geocoding
                ]
                house_infos.append(house_info)

                # Increment the house_code_counter
                house_code_counter += 1

    return house_deals, house_infos

# Process the CSV files
house_deals, house_infos = process_csv(raw_csv_files)

# Define headers for the new CSV files
house_deal_headers = [
    "deal_code", "deal_type", "deal_amount", "deposit_amount",
    "monthly_amount", "deal_year", "deal_month", "deal_day",
    "area", "floor", "house_code", "house_type"
]

house_info_headers = [
    "house_code", "house_type", "build_year", "dong_code",
    "house_name", "si_gun_gu_name","dong_name", "road_name", "bonbun",
    "bubun", "address", "lng", "lat"
]

# Save the new data to CSV files
house_deal_df = pd.DataFrame(house_deals, columns=house_deal_headers)
house_info_df = pd.DataFrame(house_infos, columns=house_info_headers)

house_deal_df.to_csv(new_house_deal_csv, index=False, encoding='utf-8')
house_info_df.to_csv(new_house_info_csv, index=False, encoding='utf-8')

In [26]:
import pandas as pd
import requests

# Define the path to the house_info CSV file
house_info_csv = 'house_info.csv'
updated_house_info_csv = 'updated_house_info.csv'

# Function to get coordinates and b_code from Kakao Local API
def get_address_info(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['documents']:
            document = data['documents'][0]
            b_code = document['address']['b_code'] if 'b_code' in document['address'] else None
            building_name = document['road_address']['building_name'] if 'road_address' in document and 'building_name' in document['road_address'] else None
            x = document['x'] if 'x' in document else None
            y = document['y'] if 'y' in document else None
            return b_code, building_name, x, y
        else:
            print(f"No documents found for address: {address}")
    else:
        print(f"Failed to get response for address: {address}, Status Code: {response.status_code}")
    return None, None, None, None

# Define your Kakao REST API Key
api_key = "826ff6e6031401a7fc4aba8410ea5565"

# Read the house_info CSV file
house_info_df = pd.read_csv(house_info_csv, encoding='utf-8')

# Initialize new columns if they don't exist
if 'dong_code' not in house_info_df.columns:
    house_info_df['dong_code'] = ''
if 'lng' not in house_info_df.columns:
    house_info_df['lng'] = ''
if 'lat' not in house_info_df.columns:
    house_info_df['lat'] = ''

# Counter for API requests
api_request_count = 0
api_request_limit = 50000

# Process each address and update the DataFrame
for index, row in house_info_df.iterrows():
    if api_request_count >= api_request_limit:
        print("API request limit reached. Stopping further requests.")
        break

    address = row['address']
    print(f"Processing address: {address}")
    
    b_code, building_name, lng, lat = get_address_info(address, api_key)
    
    if b_code and lng and lat:
        house_info_df.at[index, 'dong_code'] = b_code
        house_info_df.at[index, 'lng'] = lng
        house_info_df.at[index, 'lat'] = lat
        print(f"Updated: {address} with b_code: {b_code}, lng: {lng}, lat: {lat}")
    else:
        print(f"Could not update: {address}")

    if building_name:
        house_info_df.at[index, 'house_name'] = building_name

    # Increment the API request counter
    api_request_count += 1

# Save the updated data to a new CSV file
house_info_df.to_csv(updated_house_info_csv, index=False, encoding='utf-8')

print("Updated house_info.csv has been saved.")


Processing address: 서울특별시 성동구 옥수동 매봉길 15
Updated: 서울특별시 성동구 옥수동 매봉길 15 with b_code: 1120011300, lng: 127.013847173899, lat: 37.5460598514009
Processing address: 서울특별시 성동구 옥수동 독서당로 218
Updated: 서울특별시 성동구 옥수동 독서당로 218 with b_code: 1120011300, lng: 127.015554372439, lat: 37.5426294442931
Processing address: 서울특별시 성동구 옥수동 매봉길 50
Updated: 서울특별시 성동구 옥수동 매봉길 50 with b_code: 1120011300, lng: 127.01218936757, lat: 37.5472989077685
Processing address: 서울특별시 성동구 옥수동 독서당로40길 25


C:\Users\SSAFY\AppData\Local\Temp\ipykernel_26580\3196051173.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '127.013847173899' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  house_info_df.at[index, 'lng'] = lng
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_26580\3196051173.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '37.5460598514009' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  house_info_df.at[index, 'lat'] = lat


Updated: 서울특별시 성동구 옥수동 독서당로40길 25 with b_code: 1120011300, lng: 127.019302444208, lat: 37.5433496006445
Processing address: 서울특별시 성동구 옥수동 독서당로 191
Updated: 서울특별시 성동구 옥수동 독서당로 191 with b_code: 1120011300, lng: 127.011984339203, lat: 37.5445928680771
Processing address: 서울특별시 성동구 옥수동 독서당로 175
Updated: 서울특별시 성동구 옥수동 독서당로 175 with b_code: 1120011300, lng: 127.009810204177, lat: 37.5416160761673
Processing address: 서울특별시 성동구 옥수동 독서당로40길 37
Updated: 서울특별시 성동구 옥수동 독서당로40길 37 with b_code: 1120011300, lng: 127.017887760913, lat: 37.5418507402833
Processing address: 서울특별시 성동구 옥수동 한림말길 50
Updated: 서울특별시 성동구 옥수동 한림말길 50 with b_code: 1120011300, lng: 127.016822903142, lat: 37.5399397859799
Processing address: 서울특별시 성동구 옥수동 독서당로 242
Updated: 서울특별시 성동구 옥수동 독서당로 242 with b_code: 1120011300, lng: 127.01758830871, lat: 37.5450100498153
Processing address: 서울특별시 성동구 옥수동 독서당로 156
Updated: 서울특별시 성동구 옥수동 독서당로 156 with b_code: 1120011300, lng: 127.013812696365, lat: 37.5395083271239
Processing address: 서울특별시

TypeError: argument of type 'NoneType' is not iterable